In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
import seaborn as sns
import os

import matplotlib.dates as mdates

import scipy.stats as stats

## I'll be working with the Metro Nashville Police Department Service Calls from 2017-2024. The data is broken up into different tables by year. This is an open - ended analysis and our only constraint is a 4 hour time limit. Given the time constraint, I will be limiting my analysis to officer and resource allocation given past service calls and Time Series Analysis to predict future service calls.

In [2]:
df_2024 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2024.csv', dtype={17: str})

In [3]:
df_2024.head()

,OBJECTID,Event Number,Complaint Number,Tencode,Tencode Description,Tencode Suffix,Tencode Suffix Description,Disposition Code,Disposition Description,Block,Street Name,Unit Dispatched,Shift,Sector,Mapped Location,POINT_X,POINT_Y,Zone,Latitude,Longitude,RPA,Call Received,x,y
0,1,PD202400492222,NaN,65,DANGEROUS / INJURED ANIMAL,P,PROGRESS,11,DISREGARD / SIGNAL 9,800.0,LEMONT DR,NaN,B,231,NaN,NaN,NaN,225,NaN,NaN,1841.0,6/19/2024 8:25:38 PM,NaN,NaN
1,2,PD202400294388,NaN,70,BURGLARY - RESIDENCE,P,PROGRESS,11,DISREGARD / SIGNAL 9,NaN,NaN,820A,A,MT,NaN,NaN,NaN,827,NaN,NaN,8651.0,4/12/2024 9:48:14 AM,NaN,NaN
2,3,PD202400519980,NaN,50,THEFT,TV,NaN,5,GONE ON ARRIVAL,900.0,BUCHANAN ST,613C,C,613,NaN,NaN,NaN,611,NaN,NaN,4451.0,6/29/2024 3:11:08 AM,NaN,NaN
3,4,PD202400178239,2.024015e+10,44,DISORDERLY PERSON,P,PROGRESS,1,M.P.D. REPORT COMPLED,NaN,NaN,315B,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/3/2024 7:36:04 PM,NaN,NaN
4,5,PD202400679437,NaN,93,TRAFFIC VIOLATION,NaN,NaN,3M,NaN,NaN,MCFERRIN AVE,227A,A,EAST,NaN,NaN,NaN,221,36.192,-86.753,1903.0,8/24/2024 7:55:39 AM,-9.657300e+06,4.327073e+06


In [4]:
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279874 entries, 0 to 279873
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   OBJECTID                    279874 non-null  int64  
 1   Event Number                279874 non-null  object 
 2   Complaint Number            46549 non-null   float64
 3   Tencode                     279874 non-null  int64  
 4   Tencode Description         257311 non-null  object 
 5   Tencode Suffix              221094 non-null  object 
 6   Tencode Suffix Description  184471 non-null  object 
 7   Disposition Code            277824 non-null  object 
 8   Disposition Description     251491 non-null  object 
 9   Block                       177945 non-null  float64
 10  Street Name                 213272 non-null  object 
 11  Unit Dispatched             250262 non-null  object 
 12  Shift                       279874 non-null  object 
 13  Sector        

In [5]:
#Checking for duplicates
duplicates = df_2024[df_2024.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event Number, Complaint Number, Tencode, Tencode Description, Tencode Suffix, Tencode Suffix Description, Disposition Code, Disposition Description, Block, Street Name, Unit Dispatched, Shift, Sector, Mapped Location, POINT_X, POINT_Y, Zone, Latitude, Longitude, RPA, Call Received, x, y]
Index: []


In [6]:
#Missing every value from POINT_X & POINT_Y. Confirmed with original data source that this data is not given.
#Mapped Location is also not in original dataset and is missing every value
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2024.drop(columns=['Event Number', 'Tencode Suffix', 'Tencode Suffix Description', 'Disposition Code', 'Disposition Description',
                      'Complaint Number', 'POINT_X', 'POINT_Y','Mapped Location', 'Latitude', 'Longitude', 'OBJECTID', 'x', 'y',
                      'Unit Dispatched', 'RPA'], inplace=True)

In [7]:
column_mapping = {
    'Tencode Description': 'Tencode_Description',
    'Street Name': 'Street_Name',
    'Call Received': 'Call_Received'}

df_2024.rename(columns=column_mapping, inplace=True)

In [8]:
df_2024.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,65,DANGEROUS / INJURED ANIMAL,800.0,LEMONT DR,B,231,225,6/19/2024 8:25:38 PM
1,70,BURGLARY - RESIDENCE,NaN,NaN,A,MT,827,4/12/2024 9:48:14 AM
2,50,THEFT,900.0,BUCHANAN ST,C,613,611,6/29/2024 3:11:08 AM
3,44,DISORDERLY PERSON,NaN,NaN,B,NaN,NaN,3/3/2024 7:36:04 PM
4,93,TRAFFIC VIOLATION,NaN,MCFERRIN AVE,A,EAST,221,8/24/2024 7:55:39 AM


In [9]:
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279874 entries, 0 to 279873
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Tencode              279874 non-null  int64  
 1   Tencode_Description  257311 non-null  object 
 2   Block                177945 non-null  float64
 3   Street_Name          213272 non-null  object 
 4   Shift                279874 non-null  object 
 5   Sector               159876 non-null  object 
 6   Zone                 209733 non-null  object 
 7   Call_Received        279874 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 17.1+ MB


In [10]:
df_2023 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2023.csv', dtype={14: str})

In [11]:
df_2023.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD202300794205,NaN,88,INVESTIGATE 911 HANG-UP CALL,PW,NaN,4,ASSISTED CITIZEN,2500.0,DAVID DR,511A,A,517,517,NaN,NaN,9113.0,10/30/2023 12:52:30 PM,NaN,NaN
1,2,PD202300562586,2.023046e+10,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,10,NO RESPONSE,NaN,NaN,113A,A,NaN,NaN,NaN,NaN,NaN,8/5/2023 2:42:40 PM,NaN,NaN
2,3,PD202300121072,NaN,50,THEFT,RV,NaN,10,NO RESPONSE,300.0,TRAILS CIR,533B,B,533,533,NaN,NaN,8957.0,2/17/2023 9:45:20 PM,NaN,NaN
3,4,PD202300886771,NaN,70,BURGLARY - RESIDENCE,A,ALARM,11,DISREGARD / SIGNAL 9,600.0,CHELMSFORD PL,NaN,A,127,113,NaN,NaN,6103.0,12/4/2023 1:45:48 PM,NaN,NaN
4,5,PD202300170942,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,5,GONE ON ARRIVAL,100.0,COLEMONT CT,325C,B,333,323,NaN,NaN,8713.0,3/9/2023 4:20:06 AM,NaN,NaN


In [12]:
#Checking for duplicates
duplicates = df_2023[df_2023.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [13]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2023.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [14]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2023.rename(columns=column_mapping, inplace=True)

In [15]:
df_2023.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,88,INVESTIGATE 911 HANG-UP CALL,2500.0,DAVID DR,A,517,517,10/30/2023 12:52:30 PM
1,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,NaN,NaN,A,NaN,NaN,8/5/2023 2:42:40 PM
2,50,THEFT,300.0,TRAILS CIR,B,533,533,2/17/2023 9:45:20 PM
3,70,BURGLARY - RESIDENCE,600.0,CHELMSFORD PL,A,127,113,12/4/2023 1:45:48 PM
4,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,100.0,COLEMONT CT,B,333,323,3/9/2023 4:20:06 AM


In [16]:
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444026 entries, 0 to 444025
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Tencode              444026 non-null  int64  
 1   Tencode_Description  412752 non-null  object 
 2   Block                278570 non-null  float64
 3   Street_Name          334057 non-null  object 
 4   Shift                444026 non-null  object 
 5   Sector               254366 non-null  object 
 6   Zone                 334086 non-null  object 
 7   Call_Received        444026 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 27.1+ MB


In [17]:
df_2022 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2022.csv', dtype={14: str})

In [18]:
df_2022.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD202200451635,2.022036e+10,50,THEFT,R,REPORT,1,M.P.D. REPORT COMPLED,NaN,NaN,525B,B,NaN,NaN,NaN,NaN,NaN,7/24/2022 11:44:55 PM,NaN,NaN
1,2,PD202200630817,NaN,93,TRAFFIC VIOLATION,NaN,NaN,9T,NaN,3100.0,DICKERSON PIKE,211B,C,NaN,225,36.230,-86.759,1807.0,10/10/2022 5:38:25 AM,-86.759,36.230
2,3,PD202200445386,2.022036e+10,50,THEFT,TV,NaN,1,M.P.D. REPORT COMPLED,100.0,ARBOR KNOLL BLVD,715B,B,NaN,333,36.042,-86.668,8753.0,7/21/2022 11:39:37 PM,-86.668,36.042
3,4,PD202200299563,NaN,46,VEHICLE ACCIDENT - PERSONAL INJURY,P,PROGRESS,11,DISREGARD / SIGNAL 9,NaN,AUGUSTA DR,520B,B,523,525,36.212,-86.594,9537.0,5/17/2022 11:43:32 PM,-86.594,36.212
4,5,PD202200593226,NaN,87,SAFETY HAZARD,P,PROGRESS,NaN,NaN,NaN,CENTRAL PIKE,NaN,B,521,525,36.188,-86.634,9601.0,9/24/2022 2:56:14 AM,-86.634,36.188


In [19]:
#Checking for duplicates
duplicates = df_2022[df_2022.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [20]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2022.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [21]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2022.rename(columns=column_mapping, inplace=True)

In [22]:
df_2022.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,50,THEFT,NaN,NaN,B,NaN,NaN,7/24/2022 11:44:55 PM
1,93,TRAFFIC VIOLATION,3100.0,DICKERSON PIKE,C,NaN,225,10/10/2022 5:38:25 AM
2,50,THEFT,100.0,ARBOR KNOLL BLVD,B,NaN,333,7/21/2022 11:39:37 PM
3,46,VEHICLE ACCIDENT - PERSONAL INJURY,NaN,AUGUSTA DR,B,523,525,5/17/2022 11:43:32 PM
4,87,SAFETY HAZARD,NaN,CENTRAL PIKE,B,521,525,9/24/2022 2:56:14 AM


In [23]:
df_2021 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2021.csv', dtype={14: str})

In [24]:
df_2021.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD202100507386,2.021042e+10,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,RT,NaN,1,M.P.D. REPORT COMPLED,NaN,NaN,TE16,B,NaN,NaN,NaN,NaN,NaN,8/9/2021 10:21:07 PM,NaN,NaN
1,2,PD202100266861,NaN,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,P,PROGRESS,6,ASSISTED OTHER UNIT,700.0,MITCHELL AVE,630A,A,NaN,NaN,NaN,NaN,NaN,4/24/2021 12:22:19 PM,NaN,NaN
2,3,PD202100110057,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,4,ASSISTED CITIZEN,100.0,ARBOR KNOLL BLVD,333C,C,NaN,333,36.042,-86.668,8753.0,2/17/2021 8:30:32 AM,-86.668,36.042
3,4,PD202100112628,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,15,COMMUNITY POLICING,500.0,OAKDALE DR,6E14,A,NaN,NaN,NaN,NaN,NaN,2/18/2021 1:13:05 PM,NaN,NaN
4,5,PD202100113407,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,4,ASSISTED CITIZEN,500.0,OAKDALE DR,6E14,B,NaN,NaN,NaN,NaN,NaN,2/18/2021 9:31:27 PM,NaN,NaN


In [25]:
#Checking for duplicates
duplicates = df_2021[df_2021.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [26]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2021.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [27]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2021.rename(columns=column_mapping, inplace=True)

In [28]:
df_2021.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,NaN,NaN,B,NaN,NaN,8/9/2021 10:21:07 PM
1,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,700.0,MITCHELL AVE,A,NaN,NaN,4/24/2021 12:22:19 PM
2,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,100.0,ARBOR KNOLL BLVD,C,NaN,333,2/17/2021 8:30:32 AM
3,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,500.0,OAKDALE DR,A,NaN,NaN,2/18/2021 1:13:05 PM
4,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,500.0,OAKDALE DR,B,NaN,NaN,2/18/2021 9:31:27 PM


In [29]:
df_2020 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2020.csv', dtype={14: str})

In [30]:
df_2020.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD202000575828,NaN,43,43,P,PROGRESS,6,ASSISTED OTHER UNIT,NaN,NaN,721B,B,M,727,NaN,NaN,1753.0,8/1/2020 12:14:43 AM,NaN,NaN
1,2,PD202000470720,NaN,57,57,P,PROGRESS,11,DISREGARD / SIGNAL 9,NaN,NaN,NaN,B,H,535,NaN,NaN,8929.0,6/17/2020 3:40:48 AM,NaN,NaN
2,3,PD202000690197,NaN,43,43,P,PROGRESS,4,ASSISTED CITIZEN,NaN,NaN,7E73,B,M,727,NaN,NaN,1753.0,9/15/2020 1:47:14 AM,NaN,NaN
3,4,PD202000470737,NaN,43,43,PM,NaN,10,NO RESPONSE,NaN,NaN,520C,B,H,535,NaN,NaN,8929.0,6/17/2020 3:51:43 AM,NaN,NaN
4,5,PD202000688718,NaN,71,71,A,ALARM,13A,NaN,NaN,NaN,725A,A,M,727,NaN,NaN,1753.0,9/14/2020 2:10:13 PM,NaN,NaN


In [31]:
#Checking for duplicates
duplicates = df_2020[df_2020.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [32]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2020.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [33]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2020.rename(columns=column_mapping, inplace=True)

In [34]:
df_2020.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,43,43,NaN,NaN,B,M,727,8/1/2020 12:14:43 AM
1,57,57,NaN,NaN,B,H,535,6/17/2020 3:40:48 AM
2,43,43,NaN,NaN,B,M,727,9/15/2020 1:47:14 AM
3,43,43,NaN,NaN,B,H,535,6/17/2020 3:51:43 AM
4,71,71,NaN,NaN,A,M,727,9/14/2020 2:10:13 PM


In [35]:
df_2019 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2019.csv', dtype={14: str})

In [36]:
df_2019.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD201900437385,NaN,96,BUSINESS CHECK,NaN,NaN,12,LOCATION/BUILDING SECURE,NaN,NaN,835C,C,MT,825,NaN,NaN,8619.0,5/18/2019 10:01:15 AM,NaN,NaN
1,2,PD201900444725,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,4,ASSISTED CITIZEN,NaN,NaN,831B,B,MT,825,NaN,NaN,8619.0,5/21/2019 2:58:32 AM,NaN,NaN
2,3,PD201900324066,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,15,COMMUNITY POLICING,NaN,NaN,833B,B,MT,825,NaN,NaN,8619.0,4/12/2019 9:31:08 PM,NaN,NaN
3,4,PD201900308964,NaN,96,BUSINESS CHECK,NaN,NaN,12,LOCATION/BUILDING SECURE,NaN,NaN,833C,C,MT,825,NaN,NaN,8619.0,4/8/2019 10:44:50 AM,NaN,NaN
4,5,PD201900468305,NaN,96,BUSINESS CHECK,NaN,NaN,12,LOCATION/BUILDING SECURE,NaN,NaN,833C,C,MT,825,NaN,NaN,8619.0,5/28/2019 10:00:04 AM,NaN,NaN


In [37]:
#Checking for duplicates
duplicates = df_2019[df_2019.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [38]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2019.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [39]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2019.rename(columns=column_mapping, inplace=True)

In [40]:
df_2019.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,96,BUSINESS CHECK,NaN,NaN,C,MT,825,5/18/2019 10:01:15 AM
1,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,NaN,NaN,B,MT,825,5/21/2019 2:58:32 AM
2,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,NaN,NaN,B,MT,825,4/12/2019 9:31:08 PM
3,96,BUSINESS CHECK,NaN,NaN,C,MT,825,4/8/2019 10:44:50 AM
4,96,BUSINESS CHECK,NaN,NaN,C,MT,825,5/28/2019 10:00:04 AM


In [41]:
df_2018 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2018.csv', dtype={14: str})

In [42]:
df_2018.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD201800202497,NaN,87,SAFETY HAZARD,P,PROGRESS,11,DISREGARD / SIGNAL 9,NaN,NaN,625A,A,N,615,NaN,NaN,43020.0,2/24/2018 6:14:38 PM,NaN,NaN
1,2,PD201800176237,NaN,87,SAFETY HAZARD,P,PROGRESS,11,DISREGARD / SIGNAL 9,NaN,NaN,NaN,A,N,615,NaN,NaN,43020.0,2/17/2018 2:22:48 PM,NaN,NaN
2,3,PD201800390977,NaN,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,P,PROGRESS,10,NO RESPONSE,NaN,NaN,610B,B,N,615,NaN,NaN,43020.0,4/17/2018 8:02:09 PM,NaN,NaN
3,4,PD201800335668,NaN,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,P,PROGRESS,1,M.P.D. REPORT COMPLED,NaN,NaN,6T81,B,N,615,NaN,NaN,43020.0,4/2/2018 8:06:32 PM,NaN,NaN
4,5,PD201800701209,NaN,46,VEHICLE ACCIDENT - PERSONAL INJURY,P,PROGRESS,6,ASSISTED OTHER UNIT,NaN,NaN,625C,C,N,615,NaN,NaN,43020.0,7/15/2018 5:00:48 AM,NaN,NaN


In [43]:
#Checking for duplicates
duplicates = df_2018[df_2018.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [44]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2018.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [45]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2018.rename(columns=column_mapping, inplace=True)

In [46]:
df_2018.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,87,SAFETY HAZARD,NaN,NaN,A,N,615,2/24/2018 6:14:38 PM
1,87,SAFETY HAZARD,NaN,NaN,A,N,615,2/17/2018 2:22:48 PM
2,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,NaN,NaN,B,N,615,4/17/2018 8:02:09 PM
3,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,NaN,NaN,B,N,615,4/2/2018 8:06:32 PM
4,46,VEHICLE ACCIDENT - PERSONAL INJURY,NaN,NaN,C,N,615,7/15/2018 5:00:48 AM


In [47]:
df_2017 = pd.read_csv('../raw_data/Metro_Nashville_PD_Calls_2017.csv', dtype={14: str})

In [48]:
df_2017.head()

,OBJECTID,Event_Number,Complaint_Number,Tencode,Tencode_Description,Tencode_Suffix,Tencode_Suffix_Description,Disposition_Code,Disposition_Description,Block,Street_name,Unit_Dispatched,Shift,Sector,ZONE_,Latitude,Longitude,RPA,Call_Received,x,y
0,1,PD201701099341,NaN,96,BUSINESS CHECK,NaN,NaN,15,COMMUNITY POLICING,NaN,NaN,423C,C,C,425,NaN,NaN,4005.0,10/30/2017 9:48:42 AM,NaN,NaN
1,2,PD201700541898,NaN,96,BUSINESS CHECK,NaN,NaN,12,LOCATION/BUILDING SECURE,NaN,NaN,1F52,B,W,111,NaN,NaN,5107.0,5/29/2017 11:57:47 PM,NaN,NaN
2,3,PD201700447814,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,4,ASSISTED CITIZEN,NaN,BONNAFAIR DR,521A,A,521,525,36.213,-86.629,9603.0,5/3/2017 3:27:18 PM,-86.629,36.213
3,4,PD201700834014,NaN,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,P,PROGRESS,6T,NaN,NaN,NaN,722A,A,N,635,NaN,NaN,2212.0,8/16/2017 6:57:10 PM,NaN,NaN
4,5,PD201701281162,NaN,96,BUSINESS CHECK,NaN,NaN,15,COMMUNITY POLICING,NaN,NaN,425C,C,C,425,NaN,NaN,4005.0,12/21/2017 5:45:07 AM,NaN,NaN


In [49]:
#Checking for duplicates
duplicates = df_2017[df_2017.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [OBJECTID, Event_Number, Complaint_Number, Tencode, Tencode_Description, Tencode_Suffix, Tencode_Suffix_Description, Disposition_Code, Disposition_Description, Block, Street_name, Unit_Dispatched, Shift, Sector, ZONE_, Latitude, Longitude, RPA, Call_Received, x, y]
Index: []


In [50]:
#OBJECTID is being dropped, data is already zero indexed
#Complaint Number is a near duplicate to Event Number w/o leading 'PD'
#Latitude and Longitude are not relevant to our analysis and are being dropped
#x and y are not in original dataset but seem to correspond to Lat & Long as they are missing in the same observations so are being dropped
#Tencode Suffix and Tencode Suffix Description are descriptors of Tencode and do not add sufficient detail to the analysis being done today
df_2017.drop(columns=['OBJECTID', 'Event_Number', 'Complaint_Number', 'Tencode_Suffix', 'Tencode_Suffix_Description', 'Disposition_Code',
                 'Disposition_Description', 'Unit_Dispatched', 'Latitude', 'Longitude', 'RPA', 'x', 'y'], inplace=True)

In [51]:
#Match columns of df_2024
column_mapping = {
    'Street_name': 'Street_Name',
    'ZONE_': 'Zone'}

df_2017.rename(columns=column_mapping, inplace=True)

In [52]:
df_2017.head()

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,96,BUSINESS CHECK,NaN,NaN,C,C,425,10/30/2017 9:48:42 AM
1,96,BUSINESS CHECK,NaN,NaN,B,W,111,5/29/2017 11:57:47 PM
2,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,NaN,BONNAFAIR DR,A,521,525,5/3/2017 3:27:18 PM
3,45,VEHICLE ACCIDENT - PROPERTY DAMAGE,NaN,NaN,A,N,635,8/16/2017 6:57:10 PM
4,96,BUSINESS CHECK,NaN,NaN,C,C,425,12/21/2017 5:45:07 AM


In [53]:
df = pd.concat([df_2024, df_2023, df_2022, df_2021, df_2020, df_2019, df_2018, df_2017], ignore_index=True)

In [54]:
df

,Tencode,Tencode_Description,Block,Street_Name,Shift,Sector,Zone,Call_Received
0,65,DANGEROUS / INJURED ANIMAL,800.0,LEMONT DR,B,231,225,6/19/2024 8:25:38 PM
1,70,BURGLARY - RESIDENCE,NaN,NaN,A,MT,827,4/12/2024 9:48:14 AM
2,50,THEFT,900.0,BUCHANAN ST,C,613,611,6/29/2024 3:11:08 AM
3,44,DISORDERLY PERSON,NaN,NaN,B,NaN,NaN,3/3/2024 7:36:04 PM
4,93,TRAFFIC VIOLATION,NaN,MCFERRIN AVE,A,EAST,221,8/24/2024 7:55:39 AM
...,...,...,...,...,...,...,...,...
5226210,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,700.0,DREXEL ST,B,CENTRA,423,11/17/2017 2:59:59 AM
5226211,63,SUICIDAL PERSON,700.0,DREXEL ST,A,CENTRA,423,11/21/2017 2:44:46 PM
5226212,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,700.0,DREXEL ST,B,CENTRA,423,11/23/2017 1:07:21 AM
5226213,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,700.0,DREXEL ST,C,CENTRA,423,11/17/2017 6:23:56 AM


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5226215 entries, 0 to 5226214
Data columns (total 8 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Tencode              int64  
 1   Tencode_Description  object 
 2   Block                float64
 3   Street_Name          object 
 4   Shift                object 
 5   Sector               object 
 6   Zone                 object 
 7   Call_Received        object 
dtypes: float64(1), int64(1), object(6)
memory usage: 319.0+ MB


In [56]:
df.isnull().sum()

Tencode                      0
Tencode_Description     170558
Block                  3672206
Street_Name            3459852
Shift                        0
Sector                 1368612
Zone                    791966
Call_Received                0
dtype: int64

## I'm finding we do not have any true numerica data. Tencode is a shorthand that is used to convey messages quickly. Block refers to the city block, ie the 900 block is not "greater than" the 800 block.

## Given this discovery, I'll be converting these features to strings and naturally, will convert Call Received to datetime as well

In [59]:
# Call Received to datetime
df['Call_Received'] = pd.to_datetime(df['Call_Received'], format='%m/%d/%Y %I:%M:%S %p')

In [60]:
#Fill nulls
for column in df.columns:
    if df[column].dtype in ['float64', 'int64']:
        df[column].fillna(999999, inplace=True) #Would have filled with 0 however there is a 0 Block
    elif df[column].dtype == 'object':
        df[column].fillna('UNKNOWN', inplace=True)

In [61]:
df[['Tencode', 'Block']] = df[['Tencode', 'Block']].astype(str)

In [62]:
df['Block'] = df['Block'].str.replace('.0', '', regex=False)

In [63]:
df.isnull().sum()

Tencode                0
Tencode_Description    0
Block                  0
Street_Name            0
Shift                  0
Sector                 0
Zone                   0
Call_Received          0
dtype: int64

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5226215 entries, 0 to 5226214
Data columns (total 8 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Tencode              object        
 1   Tencode_Description  object        
 2   Block                object        
 3   Street_Name          object        
 4   Shift                object        
 5   Sector               object        
 6   Zone                 object        
 7   Call_Received        datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 319.0+ MB


In [65]:
# Save cleaned dataFrame
df.to_csv('../data/Cleaned_Metro_Nashville_PD_Calls.csv', index=False)

## With the data cleaned, I want to get an idea of what values are in each feature and then start by looking at the counts of the features.

In [ ]:
df['Tencode'].unique()

In [ ]:
df['Tencode Description'].unique()

In [ ]:
df['Disposition Code'].unique()

In [ ]:
df['Disposition Description'].unique()

In [ ]:
df['Block'].unique()

In [ ]:
df['Street Name'].unique()

In [ ]:
df['Unit Dispatched'].unique()

In [ ]:
df['Shift'].unique()

In [ ]:
df['Sector'].unique()

In [ ]:
df['Zone'].unique()

In [ ]:
df['RPA'].unique()

In [ ]:
tc_counts = df['Tencode'].value_counts()
top_20_tc = tc_counts.head(20).index
TC_top_20_df = df[df['Tencode'].isin(top_20_tc)]

# Distribution of Tencodes
plt.figure(figsize=(12,6))
TC_count = sns.countplot(y='Tencode', data=TC_top_20_df)
TC_count = plt.xlabel('Count')
TC_count = plt.ylabel('Tencode')
TC_count = plt.title('Tencode Count Plot')

In [ ]:
tc_d_counts = df['Tencode Description'].value_counts()
top_20_tc_d = tc_d_counts.head(20).index
TC_D_top_20_df = df[df['Tencode Description'].isin(top_20_tc_d)]

# Distribution of Tencode Description
plt.figure(figsize=(12,6))
TC_D_count = sns.countplot(y='Tencode Description', data=TC_D_top_20_df)
TC_D_count = plt.xlabel('Count')
TC_D_count = plt.ylabel('Tencode Description')
TC_D_count = plt.title('Tencode Description Count Plot')

## UNKNOWN seems to match almost perfectly to Tencode 3. After a quick search, I found a list of Tencodes for Nashville Metro.

### http://www.serenadedesign.com/mack/metro.htm

### Given more time, I would like to verify these codes and could potentially match the 'UNKNOWN' in Tencode Description to Tencode 3.

In [ ]:
block_counts = df['Block'].value_counts()
top_20_blocks = block_counts.head(20).index
blocks_top_20_df = df[df['Block'].isin(top_20_blocks)]

# Distribution of Tencode Description
plt.figure(figsize=(12,6))
block_count = sns.countplot(y='Block', data=blocks_top_20_df)
block_count = plt.xlabel('Count')
block_count = plt.ylabel('Block')
block_count = plt.title('Block Count Plot')

## Most of the Block data is unknown, however of the blocks that are known, the 400, 100, 300, & 600 blocks have the most service calls

In [ ]:
# Distribution of Tencode Description
plt.figure(figsize=(12,4))
shift_count = sns.countplot(y='Shift', data=df)
shift_count = plt.xlabel('Count')
shift_count = plt.ylabel('Shift')
shift_count = plt.title('Shift Count Plot')

## This will be extremely useful to us in making staffing and scheduling decisions. C clearly has the lowest amount of calls so we may want to look at our staffing during that shift and compare it to the other two shifts to determine ifwe are utilizing our resources optimally.

In [ ]:
# Extract year from 'Call Received'
df['Year'] = df['Call Received'].dt.year

# Plot
plt.figure(figsize=(14, 8))
shift_count = sns.countplot(y='Shift', hue='Year', data=df, palette='tab10')

# Set labels and title
shift_count.set_xlabel('Count')
shift_count.set_ylabel('Shift')
shift_count.set_title('Shift Count Plot by Year')

# Rotate y labels for better readability if necessary
plt.yticks(rotation=45)

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
df['Year']